# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import tensorflow as tf
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io
import time
from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from skfeature.utility import construct_W
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from skfeature.utility.sparse_learning import feature_ranking
import time
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from concrete_autoencoder import ConcreteAutoencoderFeatureSelector
from keras.layers import Dense
import pandas as pd
from skimage import io
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse
from keras.datasets import fashion_mnist
import scipy.io

#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"../Defined")
import Functions as F

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# 2. Loading data

In [2]:
data_path="../Dataset/arcene.mat"
Data = scipy.io.loadmat(data_path)

data_arr=Data['X']
label_arr_=Data['Y'][:, 0]

label_arr=np.array([0 if label_arr_i<0 else label_arr_i for label_arr_i in label_arr_])
data_arr=MinMaxScaler(feature_range=(0,1)).fit_transform(data_arr)
label_arr_onehot=label_arr

In [3]:
key_feture_number=10

# 3. Calculation

In [4]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')
    del dataframe

#--------------------------------------------------------------------------------------------------------------------------------       
def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR
 
#--------------------------------------------------------------------------------------------------------------------------------       
def cal(p_data_arr,\
        p_label_arr_onehot,\
        p_key_feture_number,\
        p_epochs_number,\
        p_seed):
    
    C_train_x,C_test_x,C_train_y,C_test_y= train_test_split(p_data_arr,p_label_arr_onehot,test_size=0.2,random_state=p_seed)

    os.environ['PYTHONHASHSEED'] = str(p_seed)
    np.random.seed(p_seed)
    rn.seed(p_seed)
    tf.compat.v1.set_random_seed(p_seed)
    
    #--------------------------------------------------------------------------------------------------------------------------------
    
    def decoder(x):
        #x = Dense(key_feture_number)(x)
        x = Dense(data_arr.shape[1])(x)
        return x

    t_start = time.time()
    selector = ConcreteAutoencoderFeatureSelector(K = p_key_feture_number, output_function = decoder, num_epochs = p_epochs_number)
    selector.fit(C_train_x, C_train_x, C_test_x, C_test_x)
    t_used=time.time() - t_start
    
    write_to_csv(np.array([t_used]),"./log"+str(key_feture_number)+"/CAE_time.csv")
    
    train_compressed_Data=p_data_arr[:, selector.get_support(indices=True)]
    
    # Classification on original features
    train_feature=C_train_x
    train_label=C_train_y
    test_feature=C_test_x
    test_label=C_test_y 
    orig_train_acc,orig_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
        
    # Classification on selected features
    C_train_selected_x,C_test_selected_x,C_train_y,C_test_y= train_test_split(train_compressed_Data,p_label_arr_onehot,test_size=0.2,random_state=p_seed)
    
    train_feature=C_train_selected_x
    train_label=C_train_y
    test_feature=C_test_selected_x
    test_label=C_test_y
    selec_train_acc,selec_test_acc=F.ETree(train_feature,train_label,test_feature,test_label,0)
    
    # Linear reconstruction
    train_feature_tuple=(C_train_selected_x,C_train_x)
    test_feature_tuple=(C_test_selected_x,C_test_x)

    reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
    
    results=np.array([orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss])
    
    write_to_csv(results.reshape(1,len(results)),"./log"+str(key_feture_number)+"/CAE_results.csv")
    
    return orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss

In [5]:
epochs_number=200

p_data_arr=data_arr
p_label_arr_onehot=label_arr_onehot
p_key_feture_number=key_feture_number
p_epochs_number=epochs_number

In [ ]:
for p_seed in np.arange(0,5):
    orig_train_acc,orig_test_acc,selec_train_acc,selec_test_acc,reconstruction_loss=cal(p_data_arr,\
                                                                                        p_label_arr_onehot,\
                                                                                        p_key_feture_number,\
                                                                                        p_epochs_number,\
                                                                                        p_seed)







Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
concrete_select (ConcreteSel (None, 10)                100001    
_________________________________________________________________
dense_1 (Dense)              (None, 10000)             110000    
Total params: 210,001
Trainable params: 210,000
Non-trainable params: 1
_________________________________________________________________
None
Train on 160 samples, validate on 40 samples
Epoch 1/200
mean max of probabilities: 0.00010325127 - temperature 10.0
160/160 [==============================] - 1s 4ms/step - loss: 0.0944 - val_loss: 0.0927
Epoch 2/200
mean max of probabilities: 0.00010325135 - temperature 9.705095
160/160 [==============================] - 0s 2ms/step - loss: 0.0862 - val_loss: 0.0845
E